# Tools and Toolchains

This notebook demonstrates using tools and toolchains when ordering with the orders api. Specifically, this notebook demonstrates the following toolchains:
 - [clip](#clip)
 - [bandmath](#bandmath)
 - [toar](#toar)
 - [composite](#composite)
 - [clip -> bandmath](#clip_bandmath)
 - [toar -> reproject -> tile](#toar_reproject_tile)

For background on ordering and downloading with the orders api, see the [Ordering and Delivery](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/orders_api_tutorials/ordering_and_delivery.ipynb) notebook.

Reference information can be found at [Tools & toolchains](https://developers.planet.com/apis/orders/tools/).

## Setup

In [ ]:
import json
import os
import pathlib
import time

import numpy as np
from planet import Auth, reporting
from planet import Session, DataClient, OrdersClient
import rasterio
from rasterio.plot import show
import requests

In [ ]:
# API Key stored as an env variable
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_API_KEY_HERE'

# construct auth tuple for use in the requests library
BASIC_AUTH = (API_KEY, '')

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

# Establish headers & Orders URL
headers = {'content-type': 'application/json'}
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

In [ ]:
# define products part of order
single_product = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    },
    {
      "item_ids": ["LC81330492015320LGN01"],
      "item_type": "Landsat8L1G",
      "product_bundle": "analytic"
    },
    
]

In [ ]:
async def poll_and_download(order):
    async with Session() as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # Grab the order ID
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            await cl.wait(order['id'], callback=bar.update_state)

        # if we get here that means the order completed. Yay! Download the files.
        filenames = await cl.download_order(order['id'])

In [ ]:
# define helpful functions for visualizing downloaded imagery
def show_rgb(img_file):
    with rasterio.open(img_file) as src:
        b,g,r,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_gray(img_file):
    with rasterio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())

## Tool Demos

### No Processing (reference)

We will order and download the unprocessed image for comparison with the output of the toolchains defined below.

In [ ]:
request = {
  "name": "no processing",
  "products": single_product,
}

In [ ]:
# allow for caching, replace this with your image file
img_file = 'data/50df6201-ea94-48f1-bec9-65dd9cd8354b/1/files/PSScene/20151119_025740_0c74/analytic_udm2/20151119_025740_0c74_3B_AnalyticMS.tif'
img_file

In [ ]:
if not os.path.isfile(img_file):
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(request)
        download = await cl.poll_and_download(order)
    img_file = next(download[d] for d in download
                    if d.endswith('_3B_AnalyticMS.tif'))
print(img_file)

### Clip
<a id='clip'></a>

Clipping is likely the most common tool that will be used. It allows us to only download the pixels we are interested in.

In [ ]:
clip_aoi = {
    "type":"Polygon",
    "coordinates":[[[94.81858044862747,15.858073043526062],
                    [94.86242249608041,15.858073043526062],
                    [94.86242249608041,15.894323164978303],
                    [94.81858044862747,15.894323164978303],
                    [94.81858044862747,15.858073043526062]]]
}

In [ ]:
# define the clip tool
clip = {
    "clip": {
        "aoi": clip_aoi
    }
}

In [ ]:
# create an order request with the clipping tool
request_clip = {
  "name": "just clip",
  "products": single_product,
  "tools": [clip]
}

request_clip

In [ ]:
# allow for caching so we don't always run clip
run_clip = True

clip_img_file = 'data/6c23f9e1-d86a-47fe-9eb2-2693196168e0/1/files/20151119_025740_0c74_3B_AnalyticMS_clip.tif'
if os.path.isfile(clip_img_file): run_clip = False

In [ ]:
if run_clip:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(request_clip)
        download = await poll_and_download(order)
    clip_img_file = next(downloaded_clip_files[d] for d in downloaded_clip_files
                     if d.endswith('_3B_AnalyticMS_clip.tif'))
clip_img_file

In [ ]:
show_rgb(img_file)
show_rgb(clip_img_file)

Alright! Clipping worked. Beautiful! This saves a lot of download bandwidth and processing on our end.

### Band Math
<a id='bandmath'></a>

To demonstrate band math we will order an NDVI image.

In [ ]:
bandmath = {
  "bandmath": {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4 + b3)"
  }
}

In [ ]:
bandmath_request = {
  "name": "band math",
  "products": single_product,
  "tools": [bandmath]
}
bandmath_request

In [ ]:
run_bandmath = True

# allow for caching so we don't always run clip
bandmath_img_file = 'data/06a733ff-3034-4ffe-b1c4-8f5d004feaff/1/files/20151119_025740_0c74_3B_AnalyticMS_bandmath.tif'
if os.path.isfile(bandmath_img_file): run_bandmath = False

In [ ]:
if run_bandmath:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(bandmath_request)
        download = await poll_and_download(order)
    bandmath_img_file = next(downloaded_bandmath_files[d] for d in downloaded_bandmath_files
                             if d.endswith('_bandmath.tif'))
bandmath_img_file

In [ ]:
show_rgb(img_file)
show_gray(bandmath_img_file)

This is a beautiful NDVI image!

### TOAR
<a id='toar'></a>

The `toar` tool converts imagery to Top of Atmosphere Reflectance.

In [ ]:
toar = {
      "toar": {
        "scale_factor": 10000
      }
}
toar_request = {
  "name": "toar",
  "products": single_product,
  "tools": [toar]
}
toar_request

In [ ]:
run_toar = True

# allow for caching so we don't always run clip
toar_img_file = 'data/d22684ab-49b5-4696-ab9c-d0a893e5c193/1/files/20151119_025740_0c74_3B_AnalyticMS_toar.tif'
if os.path.isfile(toar_img_file): run_toar = False

In [ ]:
if run_toar:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(toar_request)
        download = await poll_and_download(order)
    toar_img_file = next(downloaded_toar_files[d] for d in downloaded_toar_files
                     if d.endswith('_toar.tif'))
toar_img_file

In [ ]:
show_rgb(img_file)
show_rgb(toar_img_file)

The effect of conversion to reflectance isn't really apparent in one image. But when comparing two images taken at different times of the year, the conversion to reflectance will remove effects of the sun's location and make the images more similar.

### Composite
<a id='composite'></a>

The composite tool combines multiple images into one image, similar to mosaicing. The input images must have the same band configuration, and that band configuration will be propagated to the output image.

In [ ]:
composite = {  
   "composite":{  
   }
}

composite_request = {
  "name": "composite",
  "products": same_src_products,
  "tools": [composite]
}

composite_request

In [ ]:
run_composite = True

composite_file = 'data/be20feb1-b084-43b9-9c1c-b1d4a1ffb9fc/1/files/composite.tif'
if os.path.isfile(composite_file): run_composite = False

In [ ]:
if run_composite:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(composite_request)
        download = await poll_and_download(order)
    composite_file = next(downloaded_composite_files[d] for d in downloaded_composite_files
                          if d.endswith('composite.tif'))
composite_file

In [ ]:
show_rgb(composite_file)

Beautiful! The images have been combined into one seamless image.

### Combined Tools - Clip and Band Math
<a id='clip_bandmath'></a>

This toolchain demonstrates how we can combine the clipping tool with the NDVI band math tool to only process and download the NDVI values for pixels we are interested in. Combining tools is as simple as combining the tool definitions in a list in the order request.

In [ ]:
clip_bandmath_request = {
  "name": "clip and bandmath",
  "products": single_product,
  "tools": [clip, bandmath]
}
clip_bandmath_request

In [ ]:
run_clip_bandmath = True

clip_bandmath_file = 'data/d1a94528-c1c3-4fe2-b3e5-6ff0c340a1d6/1/files/20151119_025740_0c74_3B_AnalyticMS_clip_bandmath.tif'
if os.path.isfile(clip_bandmath_file): run_clip_bandmath = False

In [ ]:
if run_clip_bandmath:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(clip_bandmath_request)
        download = await poll_and_download(order)
    clip_bandmath_file = next(downloaded_clip_bandmath_files[d] for d in downloaded_clip_bandmath_files
                     if d.endswith('_clip_bandmath.tif'))
clip_bandmath_file

In [ ]:
show_gray(clip_bandmath_file)

It is easy to add tools into a toolchain and saves a 'ton' of unnecessary pixel downloads and processing on our end. Big win!

## TOAR, Reproject, Tile
<a id='toar_reproject_tile'></a>

For a more complicated example, we will convert the pixels to reflectance, project them to WGS84, and then tile them.

In [ ]:
reproject =     {
    "reproject": {
        "projection": "WGS84",
        "kernel": "cubic"
    }
}

tile = {
  "tile": {
    "tile_size": 1232,
    "origin_x": -180,
    "origin_y": -90,
    "pixel_size": 0.000027056277056,
    "name_template": "C1232_30_30_{tilex:04d}_{tiley:04d}"
  }
}

In [ ]:
trt_request = {
  "name": "toar reproject tile",
  "products": single_product,
  "tools": [toar, reproject, tile]
}
trt_request

In [ ]:
run_trt = True

tile_files = [
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8243_3175.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8242_3175.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8247_3178.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8243_3174.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8244_3174.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8244_3177.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8243_3176.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8247_3177.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8242_3176.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8244_3175.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8246_3177.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8245_3175.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8244_3176.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8245_3177.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8246_3178.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8245_3176.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8245_3178.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8246_3176.tif',
    'data/8482721f-7711-49f6-8e12-6874c6decb38/1/files/C1232_30_30_8246_3179.tif'
]
if os.path.isfile(tile_files[0]): run_trt = False

In [ ]:
if run_trt:
    async with Session() as sess:
        cl = OrdersClient(sess)
        order = await cl.create_order(trt_request)
        download = await poll_and_download(order)
    tile_files = list(d for d in downloaded_trt_files.values()
                     if d.name.startswith('C1232_30_30_'))
    
for f in tile_files:
    print(f)

In [ ]:
for f in tile_files[:4]:
    show_rgb(f)

In [ ]:
test_file = tile_files[0]

In [ ]:
!gdalinfo $test_file

The files have indeed been reprojected to WSG 84 and tiled. It is that easy!